In [ ]:
import os
import os.path as osp
from pathlib import Path
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
!nvidia-smi

 ### Load metadata 

In [ ]:
# Change paths accordingly
IMAGE_DIR = "changethis"
TRAIN_METADATA_PATH = "changethis"
TEST_METADATA_PATH = "changethis" 

In [ ]:
train_df = pd.read_csv(TRAIN_METADATA_PATH)
test_df = pd.read_csv(TEST_METADATA_PATH)

train_df["image_path"] = train_df.image_path.apply(
    lambda path: osp.join(IMAGE_DIR, osp.basename(path)))

test_df["image_path"] = test_df.image_path.apply(
    lambda path: osp.join(IMAGE_DIR, osp.basename(path)))

# Save updated metadata
updated_train_metadata_path = osp.join(osp.dirname(TRAIN_METADATA_PATH), Path(TRAIN_METADATA_PATH).stem + "-updated.csv")
updated_test_metadata_path = osp.join(osp.dirname(TEST_METADATA_PATH), Path(TEST_METADATA_PATH).stem + "-updated.csv")
train_df.to_csv(updated_train_metadata_path, index=False)
test_df.to_csv(updated_test_metadata_path, index=False)


### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into.

In [ ]:
os.environ["TRAIN_METADATA_PATH"] = updated_train_metadata_path 
os.environ["TEST_METADATA_PATH"] = updated_test_metadata_path 

%env CUDA_DEVICES = 0
%env WANDB_ENTITY = changethis 
%env WANDB_PROJECT = changethis 
# Optional
%env HFHUB_OWNER = changethis

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

## Train single model with a config file.

In [ ]:
!python train.py \
    --train-path $TRAIN_METADATA_PATH \
    --test-path $TEST_METADATA_PATH \
    --config-path ../configs/DF24M_224_config.yaml \
    --cuda-devices $CUDA_DEVICES \
    --wandb-entity $WANDB_ENTITY \
    --wandb-project $WANDB_PROJECT \
    --hfhub-owner $HFHUB_OWNER

## Running Sweep Training 

In [ ]:
# Change sweep configuration if needed
!wandb sweep ../sweep_configs/DF24M_224.yaml

In [ ]:
!wandb agent <WANDB_ENTITY>/<WANDB_PROJECT>...